## Expert Knowledge Worker on Restaurant 

### A question answering agent that is an expert knowledge worker
### To be used by public for restaurant recommendation.
### The agent needs to be accurate and the solution should be low cost.

This project will use RAG (Retrieval Augmented Generation) to ensure our question/answering assistant has high accuracy.


In [1]:
# imports
import os
import glob
import gradio as gr

In [2]:
# imports for langchain, plotly and Chroma

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_ollama import ChatOllama
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import plotly.graph_objects as go
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
#from langchain.embeddings import HuggingFaceEmbeddings

In [3]:
# Define some constants
MODEL = "llama3.2:1b"
db_name = "vector_db"

In [4]:
# Read in documents using LangChain's loaders
# Take everything in all the sub-folders of our knowledgebase

folders = glob.glob("knowledge-base/*")

def add_metadata(doc, doc_type):
    doc.metadata["doc_type"] = doc_type
    return doc

text_loader_kwargs = {'encoding': 'utf-8'}
# text_loader_kwargs={'autodetect_encoding': True}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.txt", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    documents.extend([add_metadata(doc, doc_type) for doc in folder_docs])

text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = text_splitter.split_documents(documents)

print(f"Total number of chunks: {len(chunks)}")
print(f"Document types found: {set(doc.metadata['doc_type'] for doc in documents)}")

Total number of chunks: 12
Document types found: {'Spice Symphony', 'The Copper Kettle', 'The Golden Spoon'}


In [5]:
# Embed each chunk using OllamaEmbedding model
# Chroma is a popular open source Vector Database based on SQLLite

embeddings = OllamaEmbeddings(model="nomic-embed-text")

# from langchain.embeddings import HuggingFaceEmbeddings
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Delete if already exists

if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

# Create vectorstore

vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 12 documents


In [6]:
# Let's investigate the vectors

collection = vectorstore._collection
count = collection.count()

sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"There are {count:,} vectors with {dimensions:,} dimensions in the vector store")

There are 12 vectors with 768 dimensions in the vector store


## Visualizing the Vector Store

Let's take a minute to look at the documents and their embedding vectors to see what's going on.

In [11]:
# Prework (with thanks to Jon R for identifying and fixing a bug in this!)

result = collection.get(include=['embeddings', 'documents', 'metadatas'])
vectors = np.array(result['embeddings'])
documents = result['documents']
metadatas = result['metadatas']
doc_types = [metadata['doc_type'] for metadata in metadatas]
colors = [['blue', 'green', 'red'][['Spice Symphony', 'The Copper Kettle', 'The Golden Spoon'].index(t)] for t in doc_types]

In [13]:
vectors.shape

(12, 768)

In [ ]:
# Visalize things in 2D using t-SNE!

tsne = TSNE(n_components=2, random_state=42,  perplexity=5)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 2D scatter plot
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='2D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x',yaxis_title='y'),
    width=800,
    height=600,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

## Create RAG with LangChain

In [15]:
# create a new Chat with Ollama
llm = ChatOllama(temperature=0.7, model=MODEL)

# Alternative way of using Ollama locally
# llm = ChatOpenAI(temperature=0.7, model_name='llama3.2', base_url='http://localhost:11434/v1', api_key='ollama')

# set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# the retriever is an abstraction over the VectorStore that will be used during RAG
retriever = vectorstore.as_retriever()

# putting it together: set up the conversation chain with the GPT 3.5 LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

C:\Users\vigne\AppData\Local\Temp\ipykernel_11120\2705703501.py:8: LangChainDeprecationWarning:

Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/



In [18]:
# Let's try a simple question

query = "Please suggest a place for Indian food lovers and list their popular menu items."
result = conversation_chain.invoke({"question": query})
print(result["answer"])

Based on the context and the quality of reviews mentioned earlier, I would highly recommend Spice Symphony as a great option for South Asian cuisine enthusiasts. The restaurant's focus on authentic Indian flavors, traditional techniques, and high-quality ingredients suggests that it would be an excellent choice for those looking to experience the rich culinary heritage of South Asia. Additionally, the warm and welcoming atmosphere described in the reviews implies that guests will feel at home during their visit.

If you're specifically interested in exploring other options, some other notable Indian restaurants in the city include:

* Karim's: Known for its modern take on traditional Indian cuisine
* Tamarind Tree: A popular spot for creative takes on South Asian flavors
* Mace Spice Market: Offers a unique blend of Indian and international flavors

However, based on the reviews and overall reputation, I believe that Spice Symphony stands out as an exceptional choice for South Asian cu

## In Gradio using the Chat interface -

A quick and easy way to prototype a chat with an LLM

In [21]:
# Wrapping that in a function

def chat(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]

In [ ]:
# And in Gradio:

view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

In [20]:
# To investigate what gets sent behind the scenes

from langchain_core.callbacks import StdOutCallbackHandler

llm = ChatOllama(temperature=0.7, model=MODEL)

memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

retriever = vectorstore.as_retriever()

conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory, callbacks=[StdOutCallbackHandler()])

query = "Please suggest a place for Indian food lovers and list their popular menu items."
result = conversation_chain.invoke({"question": query})
print("\nAnswer:", result["answer"])



> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
The menu at Spice Symphony is a carefully curated journey through the diverse regions of India, showcasing both well-known favorites and lesser-known regional specialties. Appetizers include the Samosa Chaat, a deconstructed version of the classic samosa with a medley of chutneys and yogurt, and the Tandoori Prawns, marinated in a spicy yogurt mixture and cooked to perfection in the clay oven. The main courses are where the culinary magic truly happens. Signature dishes include the rich and creamy Butter Chicken, the slow-cooked and deeply flavorful Lamb Rogan Josh, and the fiery Vindaloo, a testament to the cuisine of Goa. For vegetarians, the me